# DomoAccount

> DomoAccounts are used to provide credentials for Domo Streams.


In [70]:
# | default_exp DomoAccount

In [71]:
# | hide
from nbdev.showdoc import *

In [72]:
# | exporti
import aiohttp

import datetime as dt
import re


import nbdev_domo.DomoAuth as dmda
import nbdev_domo.ResponseGetData as rgd
import nbdev_domo.Transport as tr
import nbdev_domo.utils as utils

from fastcore.basics import patch_to
from fastcore.test import test_eq

# Domo Account Routes

## GET Routes


In [73]:
async def get_accounts(
    full_auth: dmda.DomoAuth, debug: bool = False, session: aiohttp.ClientSession = None
) -> rgd.ResponseGetData:

    url = f"https://{full_auth.domo_instance}.domo.com/api/data/v1/accounts"

    if debug:
        print(url)

    domo_transport = tr.TransportAsync(
        auth_header=await full_auth.generate_auth_header(), session=session
    )

    return await domo_transport.get(url=url)

#### sample implementation of get_accounts


In [74]:
import os
import json
import pandas as pd

creds = json.loads(os.environ["DOJO_CREDS"])

domo_auth = dmda.DomoFullAuth(
    domo_username=creds.get("domo_username"),
    domo_password=creds.get("domo_password"),
    domo_instance="domo-dojo",
)

res = await get_accounts(full_auth=domo_auth)

account_ls = res.response

test_eq(type(account_ls), list)

pd.DataFrame(account_ls[0:4])

,id,userId,name,displayName,type,valid,dataProviderType,credentialsType,createdAt,createdBy,modifiedAt,modifiedBy,configurations,accountTemplateId,accountTemplateAuthorizationId,accountId
0,1,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,dataset-copy,fields,1589100087000,1893952720,1589100087000,1893952720,{},None,None,1
1,5,1893952720,Domo Governance Datasets Third Party Account,Domo Governance Datasets Third Party Account,data,True,domo-governance-d14c2fef-49a8-4898-8ddd-f64998...,fields,1616777681000,1893952720,1616777681000,1893952720,{},None,None,5
2,27,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,domo-csv,fields,1619083568000,1893952720,1619083568000,1893952720,{},None,None,27
3,45,1893952720,onyxReporting@gmail.com,onyxReporting@gmail.com,data,True,google-spreadsheets,oauth,1664924354000,1893952720,1664924354000,1893952720,{},None,None,45


In [75]:
async def get_account_from_id(
    full_auth: dmda.DomoFullAuth,
    account_id: int,
    debug: bool = False,
    log_results: bool = False,
    session: aiohttp.ClientSession = None,
) -> rgd.ResponseGetData:
    """gets metadata about an account, does not retrieve configuration settings"""

    url = f"https://{full_auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}?unmask=true"

    if debug:
        print(url)

    domo_transport = tr.TransportAsync(
        auth_header=await full_auth.generate_auth_header(), session=session
    )

    return await domo_transport.get(url=url)

#### sample implementation of get_account_from_id


In [76]:
domo_account = account_ls[0]

res = await get_account_from_id(full_auth=domo_auth, account_id=domo_account.get("id"))

account_config = res.response
test_eq(type(account_config), dict)

pd.DataFrame([account_config])


,id,userId,name,displayName,type,valid,dataProviderType,credentialsType,createdAt,createdBy,modifiedAt,modifiedBy,configurations,accountTemplateId,accountTemplateAuthorizationId,accountId
0,1,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,dataset-copy,fields,1589100087000,1893952720,1589100087000,1893952720,{},None,None,1


In [77]:
async def get_account_config(
    full_auth: dmda.DomoAuth,
    account_id: int,
    data_provider_type: str,
    debug: bool = False,
    session: aiohttp.ClientSession = None,
) -> rgd.ResponseGetData:
    """retrieves account configuration information, does not include metadata"""

    url = f"https://{full_auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}?unmask=true"

    if debug:
        print(url)

    domo_transport = tr.TransportAsync(
        auth_header=await full_auth.generate_auth_header(), session=session
    )

    return await domo_transport.get(url=url)

#### Sample Implementation of get_account_config


In [78]:
domo_account = account_ls[0]

res = await get_account_config(
    full_auth=domo_auth,
    account_id=domo_account.get("id"),
    data_provider_type=domo_account.get("dataProviderType"),
)

account_config = res.response
test_eq(type(account_config), dict)

pd.DataFrame([account_config])

,instance,accessToken
0,northshore-io-partner.domo.com,********


## CRUD Routes

### Update


In [79]:
# | export
async def update_account_config(
    full_auth: dmda.DomoFullAuth,
    account_id: int,
    config_body: dict,  # config_body is determined by the data_provider_type
    data_provider_type: str,
    debug: bool = False,
    session: aiohttp.ClientSession = None,
) -> rgd.ResponseGetData:
    """updates account configuration.  does not alter metadata"""

    url = f"https://{full_auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}"

    if debug:
        print(url)

    domo_transport = tr.TransportAsync(
        auth_header=await full_auth.generate_auth_header(), session=session
    )

    return await domo_transport.put(
        url=url, body=config_body, debug=debug, session=session
    )

In [80]:
# | export
async def update_account_name(
    full_auth: dmda.DomoFullAuth,
    account_id: int,
    account_name: str,
    debug: bool = False,
    session: aiohttp.ClientSession = None,
) -> rgd.ResponseGetData:

    url = f"https://{full_auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}/name"

    if debug:
        print(url)

    domo_transport = tr.TransportAsync(
        auth_header=await full_auth.generate_auth_header(), session=session
    )

    return await domo_transport.put_text(
        url=url, body=account_name, session=session
    )

In [81]:
account = account_ls[0]


await update_account_name(
    full_auth = domo_auth,
    account_id = account.get('id'),
    account_name='northshore-partner-io Dataset Copy', debug=True)


https://domo-dojo.domo.com/api/data/v1/accounts/1/name


ResponseGetData(status=403, response='Forbidden', is_success=False, auth_header={'x-domo-authentication': 'eyJjdXN0b21lcklkIjoibW1tbS0wMDEyLTAyMDAiLCJleHBpcmF0aW9uIjoxNjcxNzkyODA3OTE0LCJobWFjU2lnbmF0dXJlIjoiN2EyYjk2MTIzMTM5YjMzYTBmOGU1NjEyOGU0MjQ5NjA4MjJkZWI2YWFjYWNlODUwZjcxMDgwMWJhMWQ1NjgwYSIsInJvbGUiOiJBZG1pbiIsInNpZCI6IjQwNzE1ODE4LWUwMzQtNGFhNC05MGIyLWY1NzYzMmU2Yzk1MCIsInRpbWVzdGFtcCI6MTY3MTc2NDAwNzkxNCwidG9lcyI6IlVOS05PV05TSUQiLCJ1c2VySWQiOiIxODkzOTUyNzIwIn0%3D'})

### CREATE Routes


In [82]:
# | export
async def create_account(
    full_auth: dmda.DomoFullAuth,
    config_body: dict,  # config body is dependent on data provider type
    debug: bool = False,
    log_results: bool = False,
    session: aiohttp.ClientSession = None,
)-> rgd.ResponseGetData:
    url = f"https://{full_auth.domo_instance}.domo.com/api/data/v1/accounts"

    if debug:
        print(url)

    domo_transport = tr.TransportAsync(
        auth_header=await full_auth.generate_auth_header(), session=session
    )

    return await domo_transport.post(
        url=url, body=config_body, debug=debug, session=session
    )

### DELETE Routes


In [83]:
# | export


async def delete_account(
    full_auth: dmda.DomoFullAuth,
    account_id: str,
    debug: bool = False,
    log_results: bool = False,
    session: aiohttp.ClientSession = None,
)-> rgd.ResponseGetData:

    url = (
        f"https://{full_auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}"
    )

    if debug:
        print(url)

    domo_transport = tr.TransportAsync(
        auth_header=await full_auth.generate_auth_header(), session=session
    )

    return await domo_transport.delete(url=url, debug=debug, session=session)

# DomoAccount Classes

## Specific DomoAccount Configuration Types


In [84]:
#| import hide
from abc import ABC, abstractclassmethod, abstractmethod
from typing import Optional

from dataclasses import dataclass, field
from enum import Enum


In [85]:
class DomoAccount_Config(ABC):
    @abstractclassmethod
    def _from_json():
        pass
    
    @abstractmethod
    def _to_json():
        pass

@dataclass
class DomoAccount_Config_HighBandwidthConnector(DomoAccount_Config):
    aws_access_key: str
    aws_secret_key: str = field(repr=False)
    s3_staging_dir: str
    region: str = "us-west-2"
    data_provider_type = "amazon-athena-high-bandwidth"

    @classmethod
    def _from_json(cls, obj):

        dd = utils.DictDot(obj)

        return cls(
            aws_access_key=dd.awsAccessKey,
            aws_secret_key=dd.awsSecretKey,
            s3_staging_dir=dd.s3StagingDir,
            region=dd.region,
        )

    def to_json(self):
        return {
            "awsAccessKey": self.aws_access_key,
            "awsSecretKey": self.aws_secret_key,
            "s3StagingDir": self.s3_staging_dir,
            "region": self.region,
        }


@dataclass
class DomoAccount_Config_AbstractCredential(DomoAccount_Config):
    credentials: dict

    @classmethod
    def _from_json(cls, obj):

        dd = utils.DictDot(obj)

        return cls(
            credentials=dd.credentials,
        )

    def to_json(self):
        return {"credentials": self.credentials}


### Enum of implemented account config types.

`AccountConfig` enum matches dataProviderType with a Config class.


In [86]:
class AccountConfig(Enum):
    amazon_athena_high_bandwidth = {
        "type": "amazon-athena-high-bandwidth",
        "config": DomoAccount_Config_HighBandwidthConnector,
    }

    abstract_credential_store = {
        "type": "abstract-credential-store",
        "config": DomoAccount_Config_AbstractCredential,
    }

## DomoAccount Class


In [87]:
# | export
@dataclass
class DomoAccount:
    name: str
    data_provider_type: str
    id: int = None
    created_dt: dt.datetime = None
    modified_dt: dt.datetime = None
    full_auth: dmda.DomoFullAuth = field(repr=False, default=None)

    config: AccountConfig = None

    @classmethod
    def _from_json(cls, obj: dict, full_auth: dmda.DomoFullAuth = None):

        dd = utils.DictDot(obj)

        return cls(
            id=dd.id,
            name=dd.displayName,
            data_provider_type=dd.dataProviderType,
            created_dt=utils.convert_epoch_millisecond_to_datetime(dd.createdAt),
            modified_dt=utils.convert_epoch_millisecond_to_datetime(dd.modifiedAt),
            full_auth=full_auth,
        )
    
    def config_to_json(self):
        return {
            "displayName": self.name,
            "dataProviderType": self.data_provider_type,
            "name": self.data_provider_type,
            "configurations": self.config.to_json(),
        }


In [88]:
#| exporti
@patch_to(DomoAccount, cls_method=True)
async def get_from_id(
    cls,
    full_auth: dmda.DomoAuth,
    account_id: int,
    session: aiohttp.ClientSession = None,
) -> DomoAccount:
    """
    Get account metadata and attempts to retrieve config information
    To retrieve config data, a matching DomoAccount_Config class and AccountConfig enum must exist"
    """

    account_res = await get_account_from_id(
        full_auth=full_auth, account_id=account_id, session=session
    )

    if not account_res.is_success:
        return None

    obj = account_res.response
    acc = cls._from_json(obj, full_auth)

    # get account config
    config_res = await get_account_config(
        full_auth=full_auth,
        account_id=acc.id,
        data_provider_type=acc.data_provider_type,
        session=session,
    )

    if not config_res.is_success:
        return acc

    # map account config to AccountConfig enum
    enum_clean = re.sub("-", "_", acc.data_provider_type)

    if enum_clean not in [member.name for member in AccountConfig]:
        return acc

    acc.config = (
        AccountConfig[enum_clean].value.get(
            "config")._from_json(config_res.response)
    )

    return acc

#### sample implementation of DomoAccount.get_from_id()


In [89]:
account = account_ls[0]

domo_account = await DomoAccount.get_from_id(
    full_auth=domo_auth, account_id=account.get("id")
)

test_eq(isinstance(domo_account, DomoAccount), True)

domo_account

DomoAccount(name='DataSet Copy Account', data_provider_type='dataset-copy', id=1, created_dt=datetime.datetime(2020, 5, 10, 8, 41, 27), modified_dt=datetime.datetime(2020, 5, 10, 8, 41, 27), config=None)

In [90]:
class InvalidConfigError(dmda.DomoErrror):
    """return if DomoAccount does not have a valid config attribute"""

    def __init__(
        self,
        account_id :str,
        domo_instance: str):

        message = f"invalid Config attribute for account {account_id}"

        super().__init__( message=message, domo_instance=domo_instance)


class UpdateConfigError(dmda.DomoErrror):
    """return if DomoAccount does not have a valid config attribute"""

    def __init__(
            self,
            account_id: str,
            domo_instance: str):

        message = f"failed to update Config for account {account_id}"

        super().__init__(message=message, domo_instance=domo_instance)


In [91]:
#| exporti
@patch_to(DomoAccount)
async def update_config(
    self,
    config_body : Optional[DomoAccount_Config] = None,
    full_auth: dmda.DomoFullAuth = None,
    debug: bool = False,
    session: aiohttp.ClientSession = None,
):
    if not config_body and not self.config:
        raise InvalidConfigError

    full_auth = full_auth or self.full_auth


    update_account_config_res = await update_account_config(
        full_auth=full_auth,
        account_id=self.id,
        data_provider_type=self.data_provider_type,
        config_body=self.config.to_json(),
        debug=debug,
        log_results=log_results,
        session=session,
    )

    if not update_account_config_res.is_success:
        raise UpdateConfigError

    return await self.get_from_id(full_auth=full_auth, account_id=self.id)
    
@patch_to(DomoAccount)
async def update_name(
    self,
    account_name: str = None,
    full_auth: dmda.DomoFullAuth = None,
    debug: bool = False,
    log_results: bool = False,
    session: aiohttp.ClientSession = None,
):

    full_auth = full_auth or self.full_auth

    update_account_name_res = await update_account_name(
        full_auth=full_auth,
        account_id=self.id,
        account_name=account_name or self.name,
        debug=debug,
        session=session,
    )

    if not update_account_name_res.is_success:
        print(update_account_name_res)
        raise UpdateConfigError( account_id = self.id, domo_instance= full_auth.domo_instance)
    
    return await self.get_from_id(full_auth=full_auth, account_id=self.id)

In [92]:
account = account_ls[0]

domo_account = await DomoAccount.get_from_id(
    full_auth=domo_auth, account_id=account.get("id")
)

await domo_account.update_name(account_name = 'northshore-partner-io Dataset Copy', debug = True)

https://domo-dojo.domo.com/api/data/v1/accounts/1/name
ResponseGetData(status=403, response='Forbidden', is_success=False, auth_header={'x-domo-authentication': 'eyJjdXN0b21lcklkIjoibW1tbS0wMDEyLTAyMDAiLCJleHBpcmF0aW9uIjoxNjcxNzkyODA3OTE0LCJobWFjU2lnbmF0dXJlIjoiN2EyYjk2MTIzMTM5YjMzYTBmOGU1NjEyOGU0MjQ5NjA4MjJkZWI2YWFjYWNlODUwZjcxMDgwMWJhMWQ1NjgwYSIsInJvbGUiOiJBZG1pbiIsInNpZCI6IjQwNzE1ODE4LWUwMzQtNGFhNC05MGIyLWY1NzYzMmU2Yzk1MCIsInRpbWVzdGFtcCI6MTY3MTc2NDAwNzkxNCwidG9lcyI6IlVOS05PV05TSUQiLCJ1c2VySWQiOiIxODkzOTUyNzIwIn0%3D'})


UpdateConfigError: failed to update Config for account 1 at domo-dojo

In [ ]:




async def create_account(
    self,
    full_auth: dmda.DomoFullAuth = None,
    #  config_body: dict,
    debug: bool = False,
    log_results: bool = False,
    session: aiohttp.ClientSession = None,
):
    full_auth = full_auth or self.full_auth

    res = await account_routes.create_account(
        full_auth=full_auth,
        config_body=self.config_to_json(),
        debug=debug,
        log_results=log_results,
        session=session,
    )

    if debug:
        print(res)

    if res.status != 200:
        return False

    obj = res.response
    acc = await self.get_from_id(full_auth=full_auth, account_id=obj.get("id"))
    acc.status = res.status
    acc.is_success = res.is_success
    return acc


async def delete_account(
    self,
    full_auth: dmda.DomoFullAuth = None,
    #  config_body: dict,
    debug: bool = False,
    log_results: bool = False,
    session: aiohttp.ClientSession = None,
):
    full_auth = full_auth or self.full_auth

    res = await account_routes.delete_account(
        full_auth=full_auth,
        account_id=self.id,
        debug=debug,
        log_results=log_results,
        session=session,
    )

    if debug:
        print(res)

    if res.status != 200:
        return False

    return True

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()